In [ ]:
import numpy as np
import condorgmm
import glob
from tqdm import tqdm
import condorgmm.eval.metrics
from condorgmm.utils.common.pose import Pose
from concurrent.futures import ThreadPoolExecutor
import pandas as pd

results_directory = "/home/nishadgothoskar/condorgmm/results/object_pose_tracking_ycbv_test_em__2025-03-05-02-11-29"
results_files = glob.glob(results_directory + "/*.pkl")
results_dfs = [pd.read_pickle(f) for f in results_files]
concatenated_df = pd.concat(results_dfs)

In [ ]:


se3_results_dir = condorgmm.get_root_path() / "assets" / "se3_results"
video = condorgmm.data.YCBTestVideo(48)
print(video.SCENE_NAMES)

object_names = video.YCB_MODEL_NAMES

all_results = []

max_T = 400

se3_pose_predictions = pd.DataFrame(
    columns=[
        "scene",
        "method",
        "object",
        "timestep",
        "predicted",
    ]
)

# Iterate through each object directory
for object_name in object_names:
    object_dir = se3_results_dir / object_name
    print(object_name)    
    # Iterate through each scene directory
    mid_dir = list(object_dir.glob("*"))
    scene_dirs = list(mid_dir[0].glob("*"))
    
    # Extract scene number from directory name (seqXX)

    predicted_poses = []
    gt_poses = []
    for scene_dir in scene_dirs:
        scene_num = int(scene_dir.name.split('seq')[-1])
        print(scene_num)

        video = condorgmm.data.YCBTestVideo(scene_num)
        object_id = video.YCB_MODEL_NAMES.index(object_name)
        object_index = list(video[0].object_ids).index(object_id)

        with ThreadPoolExecutor() as executor:
            predicted_poses = list(
                executor.map(
                    lambda timestep: condorgmm.Pose.from_matrix(np.loadtxt(scene_dir / f"{str(timestep).zfill(7)}.txt")).posquat,
                    range(max_T),
                )
            )
        
        new_rows = pd.DataFrame({'scene': scene_num, 'method': 'SE3-Tracknet', 'object': object_name, 'timestep': range(max_T), 'predicted': predicted_poses})
        se3_pose_predictions = pd.concat([se3_pose_predictions, new_rows], ignore_index=True)


In [ ]:
copy_of_concatenated_df = concatenated_df.copy()
copy_of_concatenated_df = copy_of_concatenated_df[copy_of_concatenated_df["metric"] == "ADD"]
copy_of_concatenated_df = copy_of_concatenated_df[["scene", "object", "timestep", "gt"]]
se3_pose_predictions_with_gt = se3_pose_predictions.merge(copy_of_concatenated_df, on=["scene", "object", "timestep"], how="right") 

In [ ]:
se3_results_df = condorgmm.eval.metrics.create_empty_results_dataframe()

object_names = condorgmm.data.YCBTestVideo.YCB_MODEL_NAMES
for scene_name in video.SCENE_NAMES:
    print(scene_name)
    video = condorgmm.data.YCBTestVideo(scene_name)
    

    object_ids = video[0].object_ids
    for object_id in object_ids:
        object_name = video.YCB_MODEL_NAMES[object_id]
        partial_df = se3_pose_predictions_with_gt[se3_pose_predictions_with_gt["scene"] == scene_name]
        partial_df = partial_df[partial_df["object"] == object_name]

        predicted_poses = partial_df["predicted"].tolist()
        gt_poses = partial_df["gt"].tolist()

        vertices = video.get_object_mesh_from_id(
            object_id
        ).vertices

        condorgmm.eval.metrics.add_object_tracking_metrics_to_results_dataframe(
            se3_results_df,
            scene_name,
            "SE3-Tracknet",
            object_name,
            [condorgmm.Pose(pose) for pose in predicted_poses],
            [condorgmm.Pose(pose) for pose in gt_poses],
            vertices,
        )

In [ ]:
se3_results_df

# Save metrics to file.
se3_results_df.to_pickle(
    condorgmm.get_root_path() / "assets" / "se3_results" / "se3_results_df_400.pkl"
)


In [ ]:
results_files = glob.glob(results_directory + "/*.pkl")
results_dfs = [pd.read_pickle(f) for f in results_files]
concatenated_df = pd.concat(results_dfs)

video = condorgmm.data.YCBTestVideo(48)

results_df = results_dfs[0]
scene, object_name = results_df.iloc[0]["scene"], results_df.iloc[0]["object"]
max_T = results_df["timestep"].max()


## Load FoundationPose results
ycb_dir = video.ycb_dir

all_foundation_pose_results_df = []
fp_loader = condorgmm.eval.fp_loader.YCBVTrackingResultLoader(
    frame_rate=1, split=ycb_dir.name
)

for scene in video.SCENE_NAMES:
    print("Scene: ", scene)
    video = condorgmm.data.YCBTestVideo(scene)

    df = fp_loader.get_dataframe(scene)
    df = df[df["timestep"] < max_T + 1]
    all_foundation_pose_results_df.append(df)

all_foundation_pose_results_df_concatenated = pd.concat(
    all_foundation_pose_results_df
)

all_se3_results_loaded = pd.read_pickle(condorgmm.get_root_path() / "assets" / "se3_results" / "se3_results_df_400.pkl")

assert len(all_foundation_pose_results_df_concatenated) == len(concatenated_df) == len(all_se3_results_loaded)


In [ ]:
# Aggregate all results
all_results_df = pd.concat(
    [concatenated_df, all_foundation_pose_results_df_concatenated, all_se3_results_loaded]
)

auc_results = all_results_df.groupby(["metric", "object", "method"])["value"].apply(
    condorgmm.eval.metrics.compute_auc
)
print(auc_results)

model_names = condorgmm.data.YCBTestVideo.YCB_MODEL_NAMES
methods = ["SE3-Tracknet", "FoundationPose", "condorgmm"]

table_string = """"""
for object_name in model_names:
    add_results = [
        auc_results["ADD"][object_name][method] * 100.0 for method in methods
    ]
    adds_results = [
        auc_results["ADD-S"][object_name][method] * 100.0 for method in methods
    ]

    # Get indices sorted by score (descending)
    add_sorted_indices = np.argsort(add_results)[::-1]
    adds_sorted_indices = np.argsort(adds_results)[::-1]

    object_name_mod = object_name.replace(r"_", r"\_")
    table_string += rf"{object_name_mod}"

    for i in range(len(methods)):
        # Color coding for ADD
        if i == add_sorted_indices[0]:
            table_string += (
                r" & \cellcolor{green}{"
                + rf"\textbf{{{add_results[i]:.1f}}}"
                + r"}"
            )
        elif i == add_sorted_indices[1]:
            table_string += (
                r" & \cellcolor{yellow}{" + rf"{add_results[i]:.1f}" + r"}"
            )
        else:
            table_string += (
                r" & \cellcolor{orange}{" + rf"{add_results[i]:.1f}" + r"}"
            )

        # Color coding for ADD-S
        if i == adds_sorted_indices[0]:
            table_string += (
                r" & \cellcolor{green}{"
                + rf"\textbf{{{adds_results[i]:.1f}}}"
                + r"}"
            )
        elif i == adds_sorted_indices[1]:
            table_string += (
                r" & \cellcolor{yellow}{" + rf"{adds_results[i]:.1f}" + r"}"
            )
        else:
            table_string += (
                r" & \cellcolor{orange}{" + rf"{adds_results[i]:.1f}" + r"}"
            )

    table_string += r"\\" + "\n"

table_string += r"\hline" + "\n"

auc_results_full = all_results_df.groupby(["metric", "method"])["value"].apply(
    condorgmm.eval.metrics.compute_auc
)
print(auc_results_full)

add_results = [auc_results_full["ADD"][method] * 100.0 for method in methods]
adds_results = [auc_results_full["ADD-S"][method] * 100.0 for method in methods]

# Get indices sorted by score (descending)
add_sorted_indices = np.argsort(add_results)[::-1]
adds_sorted_indices = np.argsort(adds_results)[::-1]

table_string += r"All Frames"

for i in range(len(methods)):
    # Color coding for ADD
    if i == add_sorted_indices[0]:
        table_string += (
            r" & \cellcolor{green}{" + rf"\textbf{{{add_results[i]:.1f}}}" + r"}"
        )
    elif i == add_sorted_indices[1]:
        table_string += r" & \cellcolor{yellow}{" + rf"{add_results[i]:.1f}" + r"}"
    else:
        table_string += r" & \cellcolor{orange}{" + rf"{add_results[i]:.1f}" + r"}"

    # Color coding for ADD-S
    if i == adds_sorted_indices[0]:
        table_string += (
            r" & \cellcolor{green}{" + rf"\textbf{{{adds_results[i]:.1f}}}" + r"}"
        )
    elif i == adds_sorted_indices[1]:
        table_string += r" & \cellcolor{yellow}{" + rf"{adds_results[i]:.1f}" + r"}"
    else:
        table_string += r" & \cellcolor{orange}{" + rf"{adds_results[i]:.1f}" + r"}"

table_string += r"\\" + "\n"

full_table_string = (
    r"""
\begin{table}[]
\centering
\caption{\textbf{Object Pose Tracking on YCB-Video Dataset}} 
\begin{tabular}{lcccccc}
\toprule
\textbf{Method}& \multicolumn{2}{c}{SE3-Tracknet} &  \multicolumn{2}{c}{FoundationPose} & \multicolumn{2}{c}{condorgmm} \\
\midrule
\textbf{Object} & ADD & ADD-S & ADD & ADD-S & ADD & ADD-S \\
\midrule
"""
    + table_string
    + r"\hline FPS & 90.1 & 30.0 & 40.1 \bottomrule \end{tabular} \end{table}"
)

tex_dir = condorgmm.get_root_path() / "condorgmm_tex"
with open(tex_dir / "ycbv_pose_tracking_table.tex", "w") as f:
    f.write(full_table_string)


In [ ]:
all_se3_results_loaded[["scene", "object", "timestep"]]

In [ ]:
concatenated_df[concatenated_df["object"] == "002_master_chef_can"]["scene"].unique()

In [ ]:
22000*2